## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-06-39-29 +0000,nyt,Australian Premier Albanese Meets China’s Xi i...,https://www.nytimes.com/2025/07/15/world/asia/...
1,2025-07-15-06-14-02 +0000,bbc,"I'm 'disappointed but not done' with Putin, Tr...",https://www.bbc.com/news/articles/c1e02q12z32o
2,2025-07-15-06-11-38 +0000,bbc,"Drivers offered up to £3,750 discount to buy e...",https://www.bbc.com/news/articles/cn5kpkypxp6o
3,2025-07-15-06-03-35 +0000,bbc,Nvidia says it will restart AI chip sales to C...,https://www.bbc.com/news/articles/cy8g22n32d0o
4,2025-07-15-04-30-50 +0000,bbc,China growth beats expectations as Trump tarif...,https://www.bbc.com/news/articles/c20r461g61xo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2376/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,11
4,china,6
29,tariffs,4
21,will,4
47,hit,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,44
32,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,36
4,2025-07-15-04-30-50 +0000,bbc,China growth beats expectations as Trump tarif...,https://www.bbc.com/news/articles/c20r461g61xo,26
8,2025-07-15-03-36-31 +0000,nyt,China’s Economy Grows Steadily Despite Trump’s...,https://www.nytimes.com/2025/07/14/business/ch...,25
10,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,23


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,44,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
32,25,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
41,18,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
40,14,2025-07-14-09-42-00 +0000,wsj,Higher Tariffs Would Likely Spur ECB to Furthe...,https://www.wsj.com/articles/higher-tariffs-wo...
23,12,2025-07-14-18-39-13 +0000,bbc,'Heart-breaking': Locals and visitors devastat...,https://www.bbc.com/news/articles/c4gd8d5jpy5o
5,9,2025-07-15-03-59-02 +0000,bbc,Meta to spend hundreds of billions to build AI...,https://www.bbc.com/news/articles/c1e02vx55wpo
24,9,2025-07-14-17-05-04 +0000,bbc,Nine killed and dozens hurt in fire at Massach...,https://www.bbc.com/news/articles/cq6mp9ydl73o
21,9,2025-07-14-18-59-30 +0000,nyt,Canada Wildfire Smoke Triggers Toronto and U.S...,https://www.nytimes.com/2025/07/14/world/canad...
42,9,2025-07-14-09-00-21 +0000,nyt,South African AIDS Activist Pushes for H.I.V. ...,https://www.nytimes.com/2025/07/14/health/sout...
19,8,2025-07-14-20-25-51 +0000,bbc,Motorists rescued as new flash flood warnings ...,https://www.bbc.com/news/videos/ce8zpggxp9eo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
